In [ ]:
import sys

sys.path.append("../")

from model.slt import SLTModel
from data.ph14t import Ph14TDataModule
from hydra import compose, initialize
import torch
import os
from bertviz import head_view


initialize(config_path="../configs")
cfg = compose("initial_train_home")
data_module = Ph14TDataModule(cfg)
data_module.setup()
model = SLTModel.load_from_checkpoint(
    "/root/shared-data/sign_language_translation_llm/outputs/epoch=epoch=55-wer=val_generate_accu=0.76.ckpt",
    cfg=cfg,
).cuda()
loader = data_module.train_dataloader()

handle = model.handles["vtg_handle"]
for batch in loader:
    with torch.no_grad():
        ids, video, video_length, text = handle.dispatch_batch(batch, model.device)
        text_ids, labels, text_length = handle.tokenize(
            text, model.tokenizer, model.device
        )
        _, atten = handle._forward(model, video, video_length, text_ids, text_length)
    break

seq_len = atten[0].shape[-1]
tokens = [f"[{i}]" for i in range(seq_len)]  # 比如：["[0]", "[1]", ..., "[11]"]



ImportError: cannot import name 'display' from 'IPython.core.display' (/opt/conda/envs/py_3.12/lib/python3.12/site-packages/IPython/core/display.py)